## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
column_names = ['Age', 'NumCompaniesWorked', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'YearsAtCompany']

# Create X_df using your selected columns
X_df = attrition_df[column_names]

# Show the data types for X_df
X_df.dtypes

,0
Age,int64
NumCompaniesWorked,int64
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
HourlyRate,int64
JobInvolvement,int64
JobLevel,int64
JobSatisfaction,int64
YearsAtCompany,int64


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
print(X_train.dtypes)
print(X_test.dtypes)

Age                        int64
NumCompaniesWorked         int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
HourlyRate                 int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
YearsAtCompany             int64
dtype: object
Age                        int64
NumCompaniesWorked         int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
HourlyRate                 int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
YearsAtCompany             int64
dtype: object


In [7]:
# Create a StandardScaler
standard_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = standard_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
d_encoder = OneHotEncoder()

# Fit the encoder to the training data
d_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_d_train = d_encoder.transform(y_train[['Department']]).toarray() # Convert to dense array
y_d_test = d_encoder.transform(y_test[['Department']]).toarray()

y_d_train

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [9]:
# Create a OneHotEncoder for the Attrition column
ohe_attrition = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
ohe_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = ohe_attrition.transform(y_train[['Attrition']])
y_test_attrition = ohe_attrition.transform(y_test[['Attrition']])

y_train_attrition

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Part 2: Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data.
column_number = len(X_train.columns)

# Create the input layer
input_layer = layers.Input(shape=(column_number,), name = "input")

# Create at least two shared layers
shared_layer_1 = layers.Dense(32, activation = "relu", name = "shared_layer_1")(input_layer)
shared_layer_2 = layers.Dense(64, activation = "relu", name = "shared_layer_2")(shared_layer_1)

In [11]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
hidden_department_layer = layers.Dense(32, activation = "relu", name = "hidden_department")(shared_layer_2)

# Create the output layer
output_department_layer = layers.Dense(3, activation = "softmax", name = "output_department")(hidden_department_layer)

In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
hidden_attrition_layer = layers.Dense(32, activation = "relu", name = "hidden_attrition")(shared_layer_2)

# Create the output layer
output_attrition_layer = layers.Dense(2, activation = "softmax", name = "output_attrition")(hidden_attrition_layer)

In [13]:
# Create the model
create_model = Model(inputs = input_layer, outputs = [output_department_layer, output_attrition_layer])

# Compile the model
create_model.compile(
    loss = {'output_department' : 'categorical_crossentropy',
            'output_attrition' : 'categorical_crossentropy'},
    optimizer = "adam",
    metrics = {'output_department' : 'accuracy',
               'output_attrition' : 'accuracy'}
)

# Summarize the model
create_model.summary()
# Summarize the model
create_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 10)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_1      │ (None, 32)        │        352 │ input[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_2      │ (None, 64)        │      2,112 │ shared_layer_1[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hidden_department   │ (None, 32)        │      2,080 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hidden_attrition    │ (None, 32)        │      2,080 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_department   │ (None, 3)         │         99 │ hidden_departmen… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_attrition    │ (None, 2)         │         66 │ hidden_attrition… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,789 (26.52 KB)

 Trainable params: 6,789 (26.52 KB)

 Non-trainable params: 0 (0.00 B)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 10)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_1      │ (None, 32)        │        352 │ input[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_2      │ (None, 64)        │      2,112 │ shared_layer_1[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hidden_department   │ (None, 32)        │      2,080 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hidden_attrition    │ (None, 32)        │      2,080 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_department   │ (None, 3)         │         99 │ hidden_departmen… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_attrition    │ (None, 2)         │         66 │ hidden_attrition… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,789 (26.52 KB)

 Trainable params: 6,789 (26.52 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Train the model
train_model = create_model.fit(X_train_scaled, {'output_department': y_d_train, 'output_attrition': y_train_attrition}, epochs=50, verbose=0)

In [15]:
# Evaluate the model with the testing data
test_model = create_model.evaluate(X_test_scaled, {'output_department': y_d_test, 'output_attrition': y_test_attrition})
test_model

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5852 - output_attrition_accuracy: 0.8308 - output_attrition_loss: 0.5775 - output_department_accuracy: 0.5807 - output_department_loss: 1.0169  


[1.5494987964630127,
 1.0204709768295288,
 0.588583767414093,
 0.8288043737411499,
 0.58423912525177]

In [16]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {test_model[3]}")
print(f"Attrition predictions accuracy: {test_model[4]}")

Department predictions accuracy: 0.8288043737411499
Attrition predictions accuracy: 0.58423912525177


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?
2. What activation functions did you choose for your output layers, and why?
3. Can you name a few ways that this model might be improved?




YOUR ANSWERS HERE

1. No, accuracy is not the best metric for this data. The 'Attrition' dataset is likely imbalanced (fewer people leave than stay), which can make accuracy misleading. Other Metrics like Precision, Recall, and F1-Score are better choices because they provide more insight into the model's performance on the minority class.

2. I chose the "softmax" activation function for both output layers. It is the standard choice for multi-class classification because it converts the model's outputs into a probability distribution across all possible classes.

3. **Feature Engineering**: we could include more of the available columns in the training data, such as ***MaritalStatus*** or ***JobRole***, after one-hot encoding them.

**Hyperparameter Tuning**: We can adjust the model's architecture by changing the number of layers or neurons, adding Dropout layers to prevent overfitting, or changing the learning rate.

**Handling Class Imbalance**: We use techniques like class weighting or resampling (e.g., SMOTE) to help the model learn the minority 'Attrition' class more effectively.